# Prompting on Liar Dataset

## Dataset Setup

In [1]:
import datasets

liar = datasets.load_dataset("liar")
liar

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 10269
    })
    test: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 1283
    })
    validation: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 1284
    })
})

In [2]:
train = liar["train"]
test = liar["test"]
val = liar["validation"]

In [3]:
full_liar = datasets.concatenate_datasets([train, test, val])
full_liar

Dataset({
    features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
    num_rows: 12836
})

## Model Loading

In [4]:
falcon = "tiiuae/falcon-7b-instruct"
llama = "meta-llama/Llama-2-7b-chat-hf"
mistral = "mistralai/Mistral-7B-Instruct-v0.2"
orca = "microsoft/Orca-2-7b"

In [5]:
# change this depending on experiment
model_name = llama

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, quantization_config=config, device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Experiment

### Experiment Utils

In [7]:
from transformers import PreTrainedTokenizer
from typing import Dict

LABEL_MAP = {
    0: "E",  # 0 : False
    1: "C",  # 1 : Half True
    2: "B",  # 2 : Mostly True
    3: "A",  # 3 : True
    4: "D",  # 4 : Barely True
    5: "F",  # 5 : Pants on Fire
}


def was_correct(
    decoded:str, entry: Dict[str, int]
) -> bool:
    return LABEL_MAP[entry["label"]] in decoded

In [8]:
n_examples = 5

In [9]:
import random
random.seed(1770)
entries = random.choices(list(range(len(train))), k=n_examples)

def to_zero_shot_prompt(entry: Dict[str, str]) -> str:
    speaker = entry["speaker"].replace("-", " ").title()
    statement = entry["statement"].lstrip("Says ")

    prompt = f"""Please select the option that most closely describes the following claim by {speaker}:\n{statement}\n\nA) True\nB) Mostly True\nC) Half True\nD) Barely True\nE) False\nF) Pants on Fire (absurd lie)\n\nChoice: ("""
    return prompt

def to_n_shot_prompt(n: int, entry: Dict[str, str]) -> str:
    examples = ""
    for i in range(n):
        examples += to_zero_shot_prompt(train[entries[i]]) + LABEL_MAP[train[entries[i]]['label']] + "\n\n"
    prompt = to_zero_shot_prompt(entry)
    return examples + prompt


In [10]:
def workflow(entry: dict, model, k:int=0, verbose: bool = False) -> bool:
    prompt = to_n_shot_prompt(k, entry)

    # encode input, move it to cuda, then generate
    encoded_input = tokenizer(prompt, return_tensors="pt")
    encoded_input = {item: val.cuda() for item, val in encoded_input.items()}
    generation = model.generate(
        **encoded_input,
        max_new_tokens=1,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

    # log the prompt and response if verbose
    if verbose:
        print(tokenizer.batch_decode(generation)[0])

    decoded = tokenizer.decode(generation[0, -1])
    correct = was_correct(decoded, entry)

    if verbose:
        print(
            "The model was",
            "correct" if correct else "incorrect",
            " - responded",
            tokenizer.decode(generation[0, -1]),
            "and answer should have been",
            LABEL_MAP[entry["label"]],
        )
    return correct

### Verify it works

In [11]:
import random
workflow(train[random.randint(0, len(train) - 1)], model, verbose=True, k=n_examples)

/home/eliot/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/eliot/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<s> Please select the option that most closely describes the following claim by Bernie S:
Today the Walton family of Walmart own more wealth than the bottom 40 percent of America.

A) True
B) Mostly True
C) Half True
D) Barely True
E) False
F) Pants on Fire (absurd lie)

Choice: (A

Please select the option that most closely describes the following claim by Kim Guadagno:
Panasonic stayed in New Jersey because of the Business Retention and Relocation Assistance Grant (BRRAG) program.

A) True
B) Mostly True
C) Half True
D) Barely True
E) False
F) Pants on Fire (absurd lie)

Choice: (E

Please select the option that most closely describes the following claim by Democratic Governors Association:
Lincoln Chafee voted with President George W. Bush and the conservative leadership 76% of the time.

A) True
B) Mostly True
C) Half True
D) Barely True
E) False
F) Pants on Fire (absurd lie)

Choice: (D

Please select the option that most closely describes the following claim by Patrick Kennedy:
"

False

### Run Experiment

Results of zero-shot prompting the models

In [ ]:
from tqdm import tqdm

num_correct = 0
answers = {}
for idx, entry in enumerate(tqdm(full_liar)):
    if idx in entries:
        continue  # don't include items that were in the examples
    
    correct = workflow(entry, model, k=n_examples)
    if correct:
        num_correct += 1

In [ ]:
# log results
with open(f"{n_examples}_shot.txt", "a") as file:
    file.write(f"{model_name} : {num_correct}/{len(full_liar)-len(entries)}\n")

In [ ]:
# print results up till now
with open(f"{n_examples}_shot.txt", "r") as file:
    print(file.read())